In [33]:
"""
Parse ANADDB output file
Currently able to parse: EO tensor, Raman tensor, polarities.
"""
from itertools import tee
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt

amu = 1.66054*(10**(-27))
# Li_mass = 6.941*amu
# Nb_mass = 92.90638*amu
# O_mass = 15.999*amu

Li_mass = 6.941
Nb_mass = 92.90638
O_mass = 15.999
### Reading the file ###
class Anaddb:
    def __init__(self, filename, natom=10):
        with open(filename, 'r') as file:
            self.data = file.readlines()
        self.natom = natom
        
    def ionic_EO(self):
        ionic_EO_tensor = np.zeros((6,3))
        index = self.data.index(" Total EO tensor (pm/V) in Voigt notations\n")+1
        
        for i in range(6):
            ionic_EO_tensor[i] = np.array(self.data[index+i].split(), dtype=float)
            
        return ionic_EO_tensor
    
    def get_EO_tensor(self):
        EO_tensors = pd.DataFrame(columns=["Mode number", "Frequency", "EO tensor"])
        index = self.data.index(" Output of the EO tensor (pm/V) in Voigt notations\n")+6
        
        for mode_number in range(4, (self.natom*3) + 1):
            ### space + digit + . + digit?
            frequency = float(re.findall("\s\d+.\d+", self.data[index])[0])
            
            EO = np.zeros((6, 3))
            for i in range(6):
                EO[i] = np.array(self.data[index+i+1].split(), dtype = float)
                
            EO_tensors.loc[len(EO_tensors.index)] = [mode_number, frequency, EO]
            index +=8
        return EO_tensors
            

    def get_Raman(self, NAC=False):
        Raman_tensors = pd.DataFrame(columns=["Mode", "Freq", "Raman susceptibility"])

        if NAC:
            index = self.data.index(" Raman susceptibility of zone-center phonons, with non-analyticity in the\n")+23
        else:
            index = self.data.index("  Raman susceptibilities of transverse zone-center phonon modes\n")+22

        for mode in np.arange(4, self.natom*3+1):
            freq = float(re.findall("\s\d+.\d+", self.data[index])[0])

            Raman = np.zeros((3,3))
            for i in range(3):
                Raman[i] = np.array(self.data[index+i+1][1:].split(), dtype=float)

            Raman_tensors.loc[len(Raman_tensors.index)] = [mode, freq, Raman]
            index += 6

        return Raman_tensors

    def get_BEC(self):
        bec = pd.DataFrame(columns=["atom", "BEC"])
        index = self.data.index(" Effective charge tensors after \n") + 4
        
        for atom in range(1, self.natom+1):
            bec_tensor = np.zeros((3, 3))
            
            for i in range(3):
                bec_tensor[i] = np.array(self.data[index+i][23:].split(), dtype=float)
            bec.loc[len(bec.index)] = [atom, bec_tensor]
            index += 3
            
        return bec        
            
        
    
    def get_polarity(self):
        polarity = pd.DataFrame(columns=["Mode", "px", "py", "pz", "|p|"])
        index = self.data.index(" Mode effective charges \n")+5
        
        for mode in np.arange(4, self.natom*3+1):
            polarity.loc[len(polarity.index)] = self.data[index][1:].split()
            index += 1

        return polarity
    
    
    def get_disp_mag(self):
        displacement = pd.DataFrame(columns=["Mode", "displacement set"])
        index = self.data.index(" Treat the first list of vectors \n")+93
        
        for mode in range(4, self.natom*3 + 1):
            disp = np.zeros((10, 3))
            
            for i in range(10):
                disp[i] = np.array(self.data[index+2*i][5:].split(), dtype=float) 
            
            disp[0] = disp[0]/np.sqrt(Li_mass)
            disp[1] = disp[1]/np.sqrt(Li_mass)
            disp[2] = disp[2]/np.sqrt(Nb_mass)
            disp[3] = disp[3]/np.sqrt(Nb_mass)
            disp[4] = disp[4]/np.sqrt(O_mass)
            disp[5] = disp[5]/np.sqrt(O_mass)
            disp[6] = disp[6]/np.sqrt(O_mass)
            disp[7] = disp[7]/np.sqrt(O_mass)
            disp[8] = disp[8]/np.sqrt(O_mass)
            disp[9] = disp[9]/np.sqrt(O_mass)
            
            
            displacement.loc[len(displacement.index)] = [mode, disp]
            index += 21
            
        return displacement    

In [37]:
"""
Post process after the extract, more human friendly print.

Be careful when using the code.
"""
if __name__ == "__main__":
    LNO = Anaddb("tnlo_4.abo")
    print(LNO.ionic_EO()*(-1))
    

def BORN(i):
    """
    The value of Born Effective Charge
    Args:
        i (integer): the index of atoms
    """
    bec = LNO.get_BEC().loc[i-1, 'BEC']
    return bec

def disp(i, j):
    """
    The magnitude of the phonon eigen displacement. 
    Args:
        i (integer): the index of modes
        j (integer): the index of atoms
    """
    disp_mag = LNO.get_disp_mag().loc[i-4, 'displacement set'][j-1:j][:].transpose()
    return disp_mag

def mode_polarity(i):
    """
    The magnitude of the mode polarity
    Args:
        i (integer): the index of modes
    """    
    mod_pol = 0
    for n in range(1, LNO.natom+1):
        mod_pol += BORN(n)@disp(i,n)
        return mod_pol.transpose() 
    
fractional_coordi = np.array([[  7.17994370124389E-01,  7.17994370124389E-01,  7.17994370124389E-01],
                              [  2.17994370124389E-01,  2.17994370124389E-01,  2.17994370124389E-01],
                              [  3.37975227703730E-04,  3.37975227703730E-04,  3.37975227703731E-04],
                              [  5.00337975227703E-01,  5.00337975227703E-01,  5.00337975227703E-01],
                              [  8.89475210181192E-01,  6.38487974670458E-01,  2.80104469786342E-01],
                              [  2.80104469786342E-01,  8.89475210181192E-01,  6.38487974670458E-01],
                              [  6.38487974670458E-01,  2.80104469786342E-01,  8.89475210181192E-01],
                              [  1.38487974670458E-01,  3.89475210181192E-01,  7.80104469786343E-01],
                              [  7.80104469786343E-01,  1.38487974670458E-01,  3.89475210181192E-01],
                              [  3.89475210181192E-01,  7.80104469786343E-01,  1.38487974670458E-01]])

[[ 1.00000000e-09 -5.52666567e+00  1.05217744e+01]
 [ 0.00000000e+00  5.52666567e+00  1.05217744e+01]
 [-1.00000000e-09  1.00000000e-09  2.82541758e+01]
 [-0.00000000e+00  1.71111434e+01  1.00000000e-09]
 [ 1.71111434e+01 -0.00000000e+00 -1.00000000e-09]
 [-5.52666567e+00  0.00000000e+00 -0.00000000e+00]]


In [48]:
# 5.47600
# np.sqrt(2.5538305001163075**2 + 1.4744547267068278**2 + 4.6046065336759421**2)
1.001472364555839*np.array([2.553830500116307, 1.4744547267068278, 4.6046065336759421, -2.9489094534136555])

array([ 2.55759067,  1.47662566,  4.61138619, -2.95325132])

In [46]:
5.476/5.467949185426249

1.001472364555839

In [49]:
# mode 4 ~ 9

for i in range(0,6):
    LNO.get_disp_mag().loc[i, 'displacement set']
    struc_perturbed = fractional_coordi + LNO.get_disp_mag().loc[i, 'displacement set']
    print("mode", i+4, "perturbed structure:")
    print(*[str(row)[1:-2] for row in struc_perturbed], sep='\n')
    print()

mode 4 perturbed structure:
0.71808931 0.71751382 0.7179943
0.21808929 0.21847492 0.2179943
0.00038635 0.00040842 0.0003379
0.50038635 0.50026753 0.5003379
0.88931363 0.63880474 0.2797048
0.27982628 0.88974069 0.6385416
0.63822367 0.28049658 0.8898211
0.13832641 0.38915843 0.7805040
0.77984018 0.13809585 0.3891293
0.38919704 0.77983898 0.1384342

mode 5 perturbed structure:
0.71847492 0.71808931 0.7179943
0.21751382 0.21808929 0.2179943
0.00026753 0.00038635 0.0003379
0.50040842 0.50038635 0.5003379
0.88914241 0.63818017 0.2802731
0.27972037 0.88928402 0.6380575
0.63823051 0.27989939 0.8897369
0.13882079 0.38916742 0.7802731
0.78036194 0.13828292 0.3897369
0.38985932 0.77991329 0.1380575

mode 6 perturbed structure:
0.71799437 0.71799437 0.7197679
0.21799437 0.21799437 0.2162207
0.00033798 0.00033798 0.0002306
0.50033798 0.50033798 0.5004453
0.88962323 0.63853036 0.2800233
0.27999375 0.88958221 0.6384068
0.63845067 0.27995509 0.8893940
0.138636   0.38943282 0.7801856
0.78006717 0.13863

In [196]:
BORN(5)

array([[-1.583855 ,  0.3240181,  0.1641891],
       [ 0.2351018, -4.04497  , -1.738459 ],
       [ 0.1357718, -1.818168 , -2.580871 ]])

In [197]:
### I don't think this is true.
mode_list = mode_polarity(4)
for i in range(5,31):        
    mode_list = np.vstack((mode_list, mode_polarity(i)))
    
np.savetxt(sys.stdout, np.abs(mode_list))


1.824093382466414571e+08 1.477472123688682747e+10 1.913571539986550046e-07
1.477472123688682747e+10 1.824093382466417551e+08 2.172611082541835202e-07
1.932873519315614903e-05 2.460125143158478471e-05 4.481050352191300201e+10
4.361762860557747841e+09 1.935882628517731094e+10 3.176369773254177995e-07
1.935882628517731094e+10 4.361762860557746887e+09 2.242563850677452056e-07
1.598003659373465070e-05 2.033909069578383627e-05 3.704709485167916870e+10
3.136751750198271751e+09 2.465415208288261795e+10 2.781215697494299020e-07
2.465415208288261795e+10 3.136751750198272228e+09 3.997297970047401533e-07
1.822738084426632043e-05 2.319946828428906112e-05 4.225719403546112061e+10
1.248002227834044845e-05 1.588433815627744695e-05 2.893288550277928543e+10
2.978068858592814255e+10 3.776957546349706268e+10 6.286537326110383909e-08
3.776957546349705505e+10 2.978068858592814255e+10 9.403318127987269879e-07
7.384205690240623062e-06 9.398478430832371433e-06 1.711907022277473259e+10
7.887676782360339165e+08 

In [42]:
disp_mag = disp(4, 1).transpose()
for n in range(2, LNO.natom+1):
    disp_mag = np.vstack((disp_mag, disp(4,n).transpose()))
np.savetxt(sys.stdout, disp_mag)


NameError: name 'sys' is not defined

In [199]:
disp_mag = disp(4, 1).transpose()
for j in range(4, 31):
    for n in range(1, LNO.natom+1):
        disp_mag = np.vstack((disp_mag, disp(j,n).transpose()))

## Normalized
disp_mag = disp_mag/np.max(disp_mag)
np.savetxt(sys.stdout, disp_mag)

5.212781514620974666e-02 -2.638424648101214687e-01 0.000000000000000000e+00
5.212781514620974666e-02 -2.638424648101214687e-01 0.000000000000000000e+00
5.211554693141186301e-02 2.638448884853681897e-01 0.000000000000000000e+00
2.655880155906457235e-02 3.867677480154391167e-02 0.000000000000000000e+00
2.656059995395380902e-02 -3.867553981485971626e-02 0.000000000000000000e+00
-8.871184640229261287e-02 1.739196354411103318e-01 -2.194003108098619981e-01
-1.527375591317873815e-01 1.457586000064141907e-01 2.947466025060482483e-02
-1.451128746170420591e-01 2.152870119539954386e-01 1.899256508337868465e-01
-8.870334984581540461e-02 -1.739274924806870071e-01 2.194046181806152407e-01
-1.451063023763898840e-01 -2.152922472350830285e-01 -1.899189687812607996e-01
-1.527277529314492455e-01 -1.457634797715600394e-01 -2.948564885029506324e-02
2.638424648101214687e-01 5.212781514620974666e-02 0.000000000000000000e+00
-2.638448884853681897e-01 5.211554693141186301e-02 0.000000000000000000e+00
-3.867677

In [200]:
disp_mag_9 = disp(9,1).transpose()
for i in range(2, 11):
    disp_mag_9 = np.vstack((disp_mag_9, disp(9, i).transpose()))

print(disp_mag_9)    
## Normalized
disp_mag_9 = disp_mag_9/np.max(disp_mag_9)
np.savetxt(sys.stdout, disp_mag_9)

[[ 0.00000000e+00  0.00000000e+00  3.59838442e+10]
 [ 0.00000000e+00  0.00000000e+00  3.59838442e+10]
 [ 0.00000000e+00  0.00000000e+00  1.36850764e+09]
 [ 0.00000000e+00  0.00000000e+00  1.36850764e+09]
 [ 1.47056871e+08  4.06741990e+09 -9.81071925e+09]
 [-3.59601740e+09 -1.90635496e+09 -9.81071925e+09]
 [ 3.44896052e+09 -2.16106494e+09 -9.81071925e+09]
 [-1.47056871e+08  4.06741990e+09 -9.81071925e+09]
 [-3.44896052e+09 -2.16106494e+09 -9.81071925e+09]
 [ 3.59601740e+09 -1.90635496e+09 -9.81071925e+09]]
0.000000000000000000e+00 0.000000000000000000e+00 1.000000000000000000e+00
0.000000000000000000e+00 0.000000000000000000e+00 1.000000000000000000e+00
0.000000000000000000e+00 0.000000000000000000e+00 3.803116853241196582e-02
0.000000000000000000e+00 0.000000000000000000e+00 3.803116853241196582e-02
4.086747099211139253e-03 1.130346127575123510e-01 -2.726423333019277750e-01
-9.993421973379426615e-02 -5.297807955041463485e-02 -2.726423333019277750e-01
9.584747254933373428e-02 -6.0056533

In [4]:
disp_mag_27 = disp(27,1).transpose()
for i in range(2, 11):
    disp_mag_27 = np.vstack((disp_mag_27, disp(27, i).transpose()))
    
## Normalized
disp_mag_27 = disp_mag_27/np.max(disp_mag_27)
np.savetxt(sys.stdout, disp_mag_27)

NameError: name 'disp' is not defined

In [202]:
mode_polarity(27)

array([[ 1.51704366e-06, -1.93086470e-06,  3.51701700e+09]])

In [203]:
polarity = LNO.get_polarity()
b = polarity.loc[:, "|p|"]
polarity

,Mode,px,py,pz,|p|
0,4,4.301493,-0.000000,-0.000000,4.301493
1,5,-0.000000,4.301493,-0.000000,4.301493
2,6,0.000000,-0.000000,0.000000,0.000000
3,7,-1.943196,0.000000,0.000000,1.943196
4,8,0.000000,-1.943196,-0.000000,1.943196
5,9,-0.000000,-0.000000,4.749020,4.749020
6,10,-3.808359,-0.000000,0.000000,3.808359
7,11,0.000000,-3.808359,0.000000,3.808359
8,12,-0.000000,-0.000000,-2.221091,2.221091
9,13,-0.000000,-0.000000,0.000000,0.000000


In [7]:
raman = LNO.get_Raman()
c = raman.loc[:, "Raman susceptibility"]

max_comp = []
for i in range(27):
    max_comp.append(np.max(np.max(np.abs(c[i:i+1]))))
# np.savetxt(sys.stdout, max_comp)
print(max_comp)


[0.004893236, 0.004893236, 0.0, 0.006870235, 0.006870235, 0.014869034, 0.002822376, 0.002822376, 0.015173458, 0.0, 0.005117305, 0.005117305, 0.002236452, 0.001760262, 0.001760262, 0.006529915, 0.006529915, 0.0, 0.004471339, 0.004471339, 0.0, 0.013030733, 0.013030733, 0.032059801, 0.001901148, 0.001901148, 0.0]


In [205]:
frequency = LNO.get_EO_tensor()
d = frequency.loc[:, "Frequency"]
d

0     148.20
1     148.20
2     207.38
3     217.55
4     217.55
5     238.21
6     253.92
7     253.92
8     260.59
9     279.99
10    309.33
11    309.33
12    335.39
13    338.11
14    338.11
15    348.69
16    348.69
17    393.59
18    417.60
19    417.60
20    441.32
21    564.38
22    564.38
23    604.74
24    656.57
25    656.57
26    871.55
Name: Frequency, dtype: float64

In [206]:
for i in range(10,31):
    for j in range(10):
        print(i)

10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
17
17
18
18
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
23
23
23
24
24
24
24
24
24
24
24
24
24
25
25
25
25
25
25
25
25
25
25
26
26
26
26
26
26
26
26
26
26
27
27
27
27
27
27
27
27
27
27
28
28
28
28
28
28
28
28
28
28
29
29
29
29
29
29
29
29
29
29
30
30
30
30
30
30
30
30
30
30


In [221]:
import numpy as np
import scipy as sci


lattice_parameter = 5.47600 # The lattice parameter of the primitive cell. a, b, and c lattice constants are the same.

fractional_coordi = np.array([[  7.17994370124389E-01,  7.17994370124389E-01,  7.17994370124389E-01],
                              [  2.17994370124389E-01,  2.17994370124389E-01,  2.17994370124389E-01],
                              [  3.37975227703730E-04,  3.37975227703730E-04,  3.37975227703731E-04],
                              [  5.00337975227703E-01,  5.00337975227703E-01,  5.00337975227703E-01],
                              [  8.89475210181192E-01,  6.38487974670458E-01,  2.80104469786342E-01],
                              [  2.80104469786342E-01,  8.89475210181192E-01,  6.38487974670458E-01],
                              [  6.38487974670458E-01,  2.80104469786342E-01,  8.89475210181192E-01],
                              [  1.38487974670458E-01,  3.89475210181192E-01,  7.80104469786343E-01],
                              [  7.80104469786343E-01,  1.38487974670458E-01,  3.89475210181192E-01],
                              [  3.89475210181192E-01,  7.80104469786343E-01,  1.38487974670458E-01]])

mode_27 = disp_mag_27

### Normalize the eigenmode with respect to the atomic mass (ref. Oxygen atoms)
# def normalize(x):
#     mode_normalized = np.copy(x)
#     ### Ba = 56, Ti = 22, O = 8
#     mode_normalized[0:1, :] = mode_normalized[0:1, :] / np.sqrt(137.3/16)
#     mode_normalized[1:2, :] = mode_normalized[1:2, :] / np.sqrt(47.8/16)
#     return mode_normalized

### We have to decrease or increase the magnitude of perturbation.

perturb = mode_27
perturbed = perturb + fractional_coordi
max_disp = np.max(perturb) * lattice_parameter
normalized_perturbed = perturb/np.max(np.abs(perturb))
normalized_max_disp = np.max(np.abs(normalized_perturbed)) * lattice_parameter

###### important parameter
amp = 0.030 # Magnitude of the amplitude in Angstrom
###### important parameter

final_perturbed = ((amp/lattice_parameter) * normalized_perturbed) + fractional_coordi
final_max_disp = np.max(np.abs((amp/lattice_parameter) * normalized_perturbed)) * lattice_parameter
# print(f"The maximum magnitude of amplitude is {max_disp} angstrom")
# print(*[str(row)[1:-2] for row in perturbed], sep='\n')
# print("")
print(f"The maximum magnitude of normalized amplitude is {final_max_disp} angstrom")
print(*[str(row)[1:-2] for row in final_perturbed], sep='\n')
print("")
print("The original fractional coordinate is :")
print(*[str(row)[1:-2] for row in fractional_coordi], sep='\n')


The maximum magnitude of normalized amplitude is 0.03 angstrom
0.71799437 0.71799437 0.7193516
0.21799437 0.21799437 0.2193516
0.00033798 0.00033798 0.0006116
0.50033798 0.50033798 0.5006116
0.8913446  0.6332413  0.2786986
0.28371353 0.89371749 0.6370821
0.63300952 0.28110887 0.8880694
0.13661858 0.38422853 0.7786986
0.78558292 0.13949237 0.3880694
0.38586615 0.78434675 0.1370821

The original fractional coordinate is :
0.71799437 0.71799437 0.7179943
0.21799437 0.21799437 0.2179943
0.00033798 0.00033798 0.0003379
0.50033798 0.50033798 0.5003379
0.88947521 0.63848797 0.2801044
0.28010447 0.88947521 0.6384879
0.63848797 0.28010447 0.8894752
0.13848797 0.38947521 0.7801044
0.78010447 0.13848797 0.3894752
0.38947521 0.78010447 0.1384879


In [216]:
mode_27

array([[ 0.        ,  0.        ,  0.2477458 ],
       [ 0.        ,  0.        ,  0.2477458 ],
       [ 0.        ,  0.        ,  0.04995393],
       [ 0.        ,  0.        ,  0.04995393],
       [ 0.34122596, -0.95769364, -0.25660336],
       [ 0.65877404,  0.77435717, -0.25660336],
       [-1.        ,  0.18333647, -0.25660336],
       [-0.34122596, -0.95769364, -0.25660336],
       [ 1.        ,  0.18333647, -0.25660336],
       [-0.65877404,  0.77435717, -0.25660336]])

In [209]:
0.294 - 0.1646

0.1294

In [222]:
p11 = 0.0292
p12 = 0.0594
p13 = 0.175
p31 = 0.147
p33 = 0.136
p44 = 0.177
p14 = -0.0901
p41 = -0.183

d51 = 67
d61 = -44
d12 = -22
d22 = 22
d42 = 67
d13 = 0.16
d23 = 0.16
d33 = 7.8

In [223]:
p11*d13 + p12*d23 + p13*d33

1.379176

In [224]:
p31*d13 + p31*d13 + p33*d33

1.10784

In [227]:
p12*d22 + p11*d22 - p14*d42

7.985899999999999

In [229]:
p41*(d12) - p41*d22 + p44*d42

19.911

In [9]:
import numpy as np

x1 = np.sqrt((5.0625167587E-01 + 9.1429057719E-03)**2 + (5.0625167587E-01 - 4.9337023257E-01)**2 + (5.1211588645E-01 -4.8685931629E-01)**2)
x2 = np.sqrt((5.0625167587E-01 - 4.9440879561E-01)**2 + (5.0625167587E-01 - 4.9440879561E-01)**2 + (5.1211588645E-01 -9.7985798757E-01)**2)

y1 = 1.0055*np.sqrt((5.09868171460310E-01 + 1.20294367917243E-02)**2 + (5.09868171460310E-01 - 4.92182438014746E-01)**2 + (5.14082330410849E-01 -4.86437120807586E-01)**2)
y2 = 1.0055*np.sqrt((5.09868171460310E-01 - 4.93490767286616E-01)**2 + (5.09868171460310E-01 - 4.93490767286616E-01)**2 + (5.14082330410849E-01 -9.77689070544885E-01)**2)

In [10]:
x2 * 7.431557746137166

0.46804185794206243

In [15]:
y2 * 7.431557746137166 * 0.529177

1.8354980494762416

In [12]:
x1

0.5161738086183444

In [16]:
y1 * 7.431557746137166 * 0.529177

2.0677839304300702

In [17]:
3.9604548145231013/3.9600231511974093

1.000109005253053

In [20]:
3.9600231511974093 * 1.011

4.003583405860581

In [19]:
3.9883728025796916/3.9449780440946540

1.010999999999999

In [21]:
3.9883675400012386/3.9449728387747149

1.0110000000000006

In [ ]:
import an